# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Collect files from directory and remote hidden files and folders from the list
for root, dirs, files in os.walk(filepath):
    # Remove Unix hidden files starting with "."
    files = [file for file in files if not file.startswith(".")]
    # Remove UNIX hidden directories
    dirs[:] = [d for d in dirs if not d[0] == '.']
    # Add files to file path list if their exstension is .csv or .CSV
    file_path_list = [file for file in files if file.endswith(".csv") or file.endswith(".CSV")]

# Join the file path and roots with the subdirectories using glob
file_path_list = glob.glob(os.path.join(root,'*'))
print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Prepare the Cassandra cluster and keyspace

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Our keyspace is named "events"
# Should it exist already then drop it
stm = "DROP KEYSPACE IF EXISTS events"
session.execute(stm)

# Now create the keyspace
stm = "CREATE KEYSPACE events WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 1};"
session.execute(stm)

#### Set Keyspace

In [7]:
# Set our current keyspace to "events", which we created above
session.execute("USE events")

## 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4




### Table preparation
- Primary key is sessionId and itemInSessions which uniquely identifies played song, artist and length
- Clustering is not required


In [8]:
# First define drop and create statement
drop1 = "DROP TABLE artistTitle_by_sessiondata"

# Define create statement
create1 = "CREATE TABLE IF NOT EXISTS artistTitle_by_sessiondata (sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"

# Try to create the table and if it exists reset the table
try:
    session.execute(create1)
except:
    session.execute(drop1)
    session.execute(create1)

### Data Upload

In [9]:
# Declare source file where we read the data from
file = 'event_datafile_new.csv'

# Open file and read content starting from line 2 (first line contains header)
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # Insert data from each line into the table
    for line in csvreader:
        # Prepare INSERT statement
        query = "INSERT INTO artistTitle_by_sessiondata (sessionId, itemInSession, artist, song_title, song_length) VALUES"
        query = query + "(%s, %s, %s, %s, %s)"
        # Run INSERT with selected colums from csv file
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]), ))

### Result Control
- Expected returned fields are artist, song_title and song_length
- Filter table for sessionId 338 and itemInSession 4 using WHERE clause

In [10]:
# First define SELECT statement
query1 = "SELECT artist, song_title, song_length FROM artistTitle_by_sessiondata WHERE sessionId = 338 and itemInSession = 4;"

# Execute statement and print results
df1 = pd.DataFrame(list(session.execute(query1)))
df1.head()

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


##  2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


### Table preparation
- Primary key is sessionId and userId since those uniquely identify the songs which a specific user listened to in a session
- Clustering key is itemInSession so that we can apply ORDER BY

In [11]:
# First define drop and create statement
drop2 = "DROP TABLE artistTitle_by_userSession"
# Define create statement
create2 = "CREATE TABLE IF NOT EXISTS artistTitle_by_userSession (sessionId int, userId int, itemInSessions int, artist text, song_title text, user_first text, user_last text, PRIMARY KEY ((sessionId, userId), itemInSessions))"

# Create table or drop and recreate
try:
    session.execute(create2)
except:
    session.execute(drop2)
    session.execute(create2)

### Data Upload

In [12]:
# Insert data from file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # Iterate each line and fill INSERT statement with data
    for line in csvreader:
        query = "INSERT INTO artistTitle_by_userSession (sessionId, userId, itemInSessions, artist, song_title, user_first, user_last) VALUES"
        query = query + "(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4]), ))

### Result control   
- Expected returned fields are artist, song_title, user_first and user_last
- Filter table for userId 10 and sessionId 182 using WHERE clause
- Order results by itemInSessions using ORDER BY

In [13]:
# First define SELECT statement
query2 = "SELECT artist, song_title, user_first, user_last FROM artistTitle_by_userSession WHERE sessionId = 182 and userId = 10 ORDER BY itemInSessions;"

# Execute query and store results in "rows" list, then iterate the list to print each line
df2 = pd.DataFrame(list(session.execute(query2)))
df2.head()

,artist,song_title,user_first,user_last
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## 3. Give first and last names of users who listened to a specific title

### Table preparation
- Primary key is composed of song_title and userId
- So we can select all rows with song_title = 'xyz' and then group the data by userId

In [14]:
# Define drop statement
drop3 = "DROP TABLE user_by_title"
# Create statement
#create3 = "CREATE TABLE IF NOT EXISTS user_by_title (song_title text, sessionId int, itemInSessions int, artist text, song_length float, userId int, user_first text, user_last text, PRIMARY KEY ((song_title), sessionId, itemInSessions))"
create3 = "CREATE TABLE IF NOT EXISTS user_by_title (song_title text, sessionId int, itemInSessions int, artist text, song_length float, userId int, user_first text, user_last text, PRIMARY KEY ((song_title), userId))"

# Create Table or drop and create again if exists
try:
    session.execute(create3)
except:
    session.execute(drop3)
    session.execute(create3)

### Data Upload   

In [15]:
# Insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # Take required fields from each line and use as input for INSERT statement
    for line in csvreader:
        query = "INSERT INTO user_by_title (song_title, sessionId, itemInSessions, artist, song_length, userId, user_first, user_last) VALUES"
        query = query + "(%s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (str(line[9]), int(line[8]), int(line[3]), str(line[0]), float(line[5]), int(line[10]), str(line[1]), str(line[4]), ))

### Result Control
- Expected query results are user_first and user_last
- Filter table for song title using WHERE clause
- Group by userId without any aggregate will lead to only the first result of each userId being selected and returned (no duplicates per user)

In [16]:
# Prepare SELECT statement
#title = "All Hands Against His Own"
title = "Fix You"
query3 = "SELECT user_first, user_last FROM user_by_title WHERE song_title = %s GROUP BY userId;"

# Execute SELECT and print query result
df3 = pd.DataFrame(list(session.execute(query3, [title])))
df3.head()          

,user_first,user_last
0,Chloe,Cuevas
1,Emily,Benson
2,Colm,Santana
3,Kate,Harrell


## Drop the tables before closing out the sessions

### Drop tables
We have already prepared the drop statements above to reset tables with each notebook run
So we simply rerun the drop statements

In [17]:
session.execute(drop1)
session.execute(drop2)
session.execute(drop3)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()